In [47]:
class TF_PowerLaw:
    def __init__(self, mode, P1, P2, P3 = [], P4 = [], P5 = [], P6 = []):
        import numpy as np
        from scipy import stats

        # Variables and Operation Mode
        self.mode = mode
        if self.mode == 'all':
            Tcte_TF = P1
            Tcte_stress = P2
            Tcte = P3
            Scte_TF = P4
            Scte_temp = P5
            Scte = P6
        elif self.mode == 'csi':
            Tcte_TF = P1
            Tcte_stress = P2
        elif self.mode == 'temp':
            Scte_TF = P1
            Scte_temp = P2
        else:
            return

        
        
        self.k = 8.617*10e-5 #Boltzmann constant => eV/K
        self.Q = 'not calculated'
        self.Ao = 'not calculated'
        self.n = 'not calculated'

        if self.mode != 'temp':
            self.x_n = []; self.y_n = []
            for i in range(0,len(Tcte_TF)):
                self.x_n.append(np.log(Tcte_stress[i]))
                self.y_n.append(np.log(Tcte_TF[i]))
    
            # Statistics
            slope, intercept, self.r_value_n, p_value, self.std_err_n = stats.linregress(self.x_n, self.y_n)
            # Numpy
            self.a_n, self.b_n = np.polyfit(np.array(self.x_n), np.array(self.y_n), deg=1)
            self.n = -self.a_n
            
            
        if self.mode != 'csi':
            self.x_Q = []; self.y_Q = []
            for i in range(0,len(Scte_TF)):
                self.x_Q.append(1/Scte_temp[i])
                self.y_Q.append(np.log(Scte_TF[i]))
    
            # Statistics
            slope, intercept, self.r_value_Q, p_value, self.std_err_Q = stats.linregress(self.x_Q, self.y_Q)
            # Numpy
            self.a_Q, self.b_Q = np.polyfit(np.array(self.x_Q), np.array(self.y_Q), deg=1)
            self.Q = self.k*self.a_Q
        
        
        if self.mode == 'all':
            self.Ao = np.exp(self.b_n - (self.Q/(self.k*Tcte)))
        
        return
    
    # --------------------------------------------------------
    # CONSTANTS
    # --------------------------------------------------------
    def Variables(self):
        string_var = "-----\nLinear Fit Variables - Power Law\n\n"
        
        if self.mode != 'temp':         

            string_var += "Finding N exponent\n" + \
            "a (slope coef) = " + str(self.a_n)+ "\n"+ \
            "b (intercept coef) = " + str(self.b_n)+ "\n"+ \
            "Correlation coefficient = " + str(self.r_value_n)+ "\n"+ \
            "Standard error = " + str(self.std_err_n)+ "\n\n"
        
        if self.mode != 'csi':         

            string_var += "Finding Activation Energy (Q)\n" + \
            "a (slope coef) = " + str(self.a_Q)+ "\n"+ \
            "b (intercept coef) = " + str(self.b_Q)+ "\n"+ \
            "Correlation coefficient = " + str(self.r_value_Q)+ "\n"+ \
            "Standard error = " + str(self.std_err_Q)+ "\n\n\n"
        
        string_var += "-----\nTime to Failure Variables - Power Law\n"
        string_var += "n = " + str(self.n)+ "\n"+ \
        "Q = " + str(self.Q)+ " [eV]\n"+ \
        "Ao = " + str(self.Ao)+ "\n\n"
        
        
        return string_var
    
    # --------------------------------------------------------
    # String para escrita da Lei de Tempo de Falha - Power Law.
    # decimal: numero de casas decimais das constantes
    # --------------------------------------------------------
    def Law(self, decimal = 3):
        import numpy as np
        
        d = '.'+str(decimal)+'e'
        
        if self.mode == 'all':
            return "TF(csi, T) = " + format(self.Ao, d) + ".(csi)^(-" + format(self.n, d) + ").exp(" + format(self.Q/self.k, d) + "/T)" 
        
        elif self.mode == 'temp':
            return "TF(T) = " + format(np.exp(self.b_Q), d) + ".exp(" + format(self.Q/self.k, d) + "/T)" 
        
        elif self.mode == 'csi':
            return "TF(csi) = " + format(np.exp(self.b_n), d) + ".(csi)^(-" + format(self.n, d) + ")" 
    
    
    # --------------------------------------------------------
    # Funcao para encontrar o TF baseada na POWER-LAW
    # TF(qsi, T) = Ao.(qsi^-n).exp(Q/kT)
    # --------------------------------------------------------
    def Find_TF(self, P1, P2=[]):
        import numpy as np
        
        # Operation Mode
        if self.mode == 'all':
            csi = P1
            T = P2
            return self.Ao*np.power(csi,-self.n)*np.exp(self.Q/(self.k*T))
        
        elif self.mode == 'temp':
            T = P1
            return np.exp(self.b_Q)*np.exp(self.Q/(self.k*np.array(T)))
            
        elif self.mode == 'csi':
            csi = P1
            return np.exp(self.b_n)*np.power(csi,-self.n)
        

       
    def LinearData_n(self):
        return self.x_n,self.y_n
    
    def LinearFit_n(self):
        self.LinearFit_n = []
        for i in range (len(self.x_n)):
            self.LinearFit_n.append(self.a_n*self.x_n[i] + self.b_n)
        return self.LinearFit_n  
    
    def LinearData_Q(self):
        return self.x2,self.y2
    
    def LinearFit_Q(self):
        self.LinearFit_Q = []
        for i in range (len(self.x_Q)):
            self.LinearFit_Q.append(self.a_Q*self.x_Q[i] + self.b_Q)
        return self.LinearFit_Q  

In [ ]:
class TF_ExponentialLaw:
    def __init__(self, mode, P1, P2, P3 = [], P4 = [], P5 = [], P6 = []):
        import numpy as np
        from scipy import stats

        # Variables and Operation Mode
        self.mode = mode
        if self.mode == 'all':
            Tcte_TF = P1
            Tcte_stress = P2
            Tcte = P3
            Scte_TF = P4
            Scte_temp = P5
            Scte = P6
        elif self.mode == 'csi':
            Tcte_TF = P1
            Tcte_stress = P2
        elif self.mode == 'temp':
            Scte_TF = P1
            Scte_temp = P2
        else:
            return       
        
        self.k = 8.617*10e-5 #Boltzmann constant => eV/K
        self.Q = 'not calculated'
        self.Ao = 'not calculated'
        self.n = 'not calculated'

        if self.mode != 'temp': #If csi is a variable or csi and temperature are both variables
            #self.x_n = []; self.y_n = []
            #for i in range(0,len(Tcte_TF)):
            self.x_n = np.array(Tcte_stress[i])
            self.y_n = np.log(Tcte_TF[i])
    
            # Statistics
            slope, intercept, self.r_value_n, p_value, self.std_err_n = stats.linregress(self.x_n, self.y_n)
            # Numpy
            self.a_n, self.b_n = np.polyfit(np.array(self.x_n), np.array(self.y_n), deg=1)
            self.n = -self.a_n
            
            
        if self.mode != 'csi': #If temperature is a variable or csi and temperature are both variables
            self.x_Q = []; self.y_Q = []
            for i in range(0,len(Scte_TF)):
                self.x_Q.append(1/Scte_temp[i])
                self.y_Q.append(np.log(Scte_TF[i]))
    
            # Statistics
            slope, intercept, self.r_value_Q, p_value, self.std_err_Q = stats.linregress(self.x_Q, self.y_Q)
            # Numpy
            self.a_Q, self.b_Q = np.polyfit(np.array(self.x_Q), np.array(self.y_Q), deg=1)
            self.Q = self.k*self.a_Q
        
        
        if self.mode == 'all':
            self.Ao = np.exp(self.b_n - (self.Q/(self.k*Tcte)))
        
        return
    
    # --------------------------------------------------------
    # CONSTANTS
    # --------------------------------------------------------
    def Variables(self):
        string_var = "-----\nLinear Fit Variables - Power Law\n\n"
        
        if self.mode != 'temp':         

            string_var += "Finding N exponent\n" + \
            "a (slope coef) = " + str(self.a_n)+ "\n"+ \
            "b (intercept coef) = " + str(self.b_n)+ "\n"+ \
            "Correlation coefficient = " + str(self.r_value_n)+ "\n"+ \
            "Standard error = " + str(self.std_err_n)+ "\n\n"
        
        if self.mode != 'csi':         

            string_var += "Finding Activation Energy (Q)\n" + \
            "a (slope coef) = " + str(self.a_Q)+ "\n"+ \
            "b (intercept coef) = " + str(self.b_Q)+ "\n"+ \
            "Correlation coefficient = " + str(self.r_value_Q)+ "\n"+ \
            "Standard error = " + str(self.std_err_Q)+ "\n\n\n"
        
        string_var += "-----\nTime to Failure Variables - Power Law\n"
        string_var += "n = " + str(self.n)+ "\n"+ \
        "Q = " + str(self.Q)+ " [eV]\n"+ \
        "Ao = " + str(self.Ao)+ "\n\n"
        
        
        return string_var
    
    # --------------------------------------------------------
    # String para escrita da Lei de Tempo de Falha - Power Law.
    # decimal: numero de casas decimais das constantes
    # --------------------------------------------------------
    def Law(self, decimal = 3):
        import numpy as np
        
        d = '.'+str(decimal)+'e'
        
        if self.mode == 'all':
            return "TF(csi, T) = " + format(self.Ao, d) + ".(csi)^(-" + format(self.n, d) + ").exp(" + format(self.Q/self.k, d) + "/T)" 
        
        elif self.mode == 'temp':
            return "TF(T) = " + format(np.exp(self.b_Q), d) + ".exp(" + format(self.Q/self.k, d) + "/T)" 
        
        elif self.mode == 'csi':
            return "TF(csi) = " + format(np.exp(self.b_n), d) + ".(csi)^(-" + format(self.n, d) + ")" 
    
    
    # --------------------------------------------------------
    # Funcao para encontrar o TF baseada na POWER-LAW
    # TF(qsi, T) = Ao.(qsi^-n).exp(Q/kT)
    # --------------------------------------------------------
    def Find_TF(self, P1, P2=[]):
        import numpy as np
        
        # Operation Mode
        if self.mode == 'all':
            csi = P1
            T = P2
            return self.Ao*np.power(csi,-self.n)*np.exp(self.Q/(self.k*T))
        
        elif self.mode == 'temp':
            T = P1
            return np.exp(self.b_Q)*np.exp(self.Q/(self.k*np.array(T)))
            
        elif self.mode == 'csi':
            csi = P1
            return np.exp(self.b_n)*np.power(csi,-self.n)
        

       
    def LinearData_n(self):
        return self.x_n,self.y_n
    
    def LinearFit_n(self):
        self.LinearFit_n = []
        for i in range (len(self.x_n)):
            self.LinearFit_n.append(self.a_n*self.x_n[i] + self.b_n)
        return self.LinearFit_n  
    
    def LinearData_Q(self):
        return self.x2,self.y2
    
    def LinearFit_Q(self):
        self.LinearFit_Q = []
        for i in range (len(self.x_Q)):
            self.LinearFit_Q.append(self.a_Q*self.x_Q[i] + self.b_Q)
        return self.LinearFit_Q  

In [41]:
class PowerLaw:
    def __init__(self, t, S, deltaT = 0):
        import numpy as np
        from scipy import stats
        self.S = S
        self.t = t
        self.deltaT = deltaT
        if self.deltaT:
            for i in range (0,len(self.t)):
                self.t[i] -= self.deltaT
        
        # ----- Linearizacao dos Dados -----
        self.lin_S = []
        self.lin_t = []
        for i in range (1,len(self.S)):
            if self.S[0] > self.S[1]:
                self.lin_S.append(np.log((self.S[0]-self.S[i])/self.S[0]))
            else:
                self.lin_S.append(np.log((self.S[i]-self.S[0])/self.S[0]))
            self.lin_t.append(np.log(self.t[i]))

            
        # ----- LINEAR FIT -----

        # Biblioteca Estatistica
        slope, intercept, self.r_value, p_value, self.std_err = stats.linregress(self.lin_t, self.lin_S)
        # Biblioteca Numpy
        self.a, self.b = np.polyfit(np.array(self.lin_t), np.array(self.lin_S), deg=1)

        self.Ao = np.exp(self.b)
        self.n = self.a
        if self.S[0] > self.S[1]:
            self.Ao = -self.Ao

        if self.deltaT:
            for i in range (0,len(self.t)):
                self.t[i] += self.deltaT
        return
    
    
    # --------------------------------------------------------
    # Funcao para encontrar o tempo de um determinado acontecimento baseada na POWER-LAW
    # S = So(1+Ao.tˆn) ----> t = (((S/So)-1)/Ao)^(1/n)
    # --------------------------------------------------------
    def Find_t(self, S):
        import numpy as np
        return np.power((((np.array(S)/self.S[0])-1)/self.Ao),(1/self.n)) + self.deltaT
    
    # --------------------------------------------------------
    # Funcao para encontrar o S baseada na POWER-LAW
    # S(t) = So(1+Ao.tˆn)
    # --------------------------------------------------------
    def Find_S(self, t):
        import numpy as np
        S = self.S[0]*(1 + self.Ao*(np.power(np.array(t)-self.deltaT,self.n)))
        return S
    
    
    # --------------------------------------------------------
    # LEI DE POTENCIA
    # S = So(1-Ao.tˆn)
    # Linearizacao
    # ln((S-So)/So) = ln(Ao) + m.ln(t)
    # y = b + a.x
    # --------------------------------------------------------
    def LinearData(self):
        return self.lin_t,self.lin_S
    
    def LinearFit(self):
        self.LinearFit = []
        for i in range (len(self.lin_t)):
            self.LinearFit.append(self.a*self.lin_t[i] + self.b)
        return self.LinearFit
    

    
    
    # --------------------------------------------------------
    # String para escrita da Power Law.
    # decimal: numero de casas decimais das constantes Ao e n
    # --------------------------------------------------------
    def LawComplete(self, decimal = 3):
        d = '.'+str(decimal)+'e'
        if self.deltaT:
            if self.Ao > 0:
                return str(self.S[0]) + " * (1 + " + format(self.Ao, d) + " * (t - " + str(self.deltaT) + ")^" + format(self.n, d) + ")"
            else:
                return str(self.S[0]) + " * (1 - " + format(-self.Ao, d) + " * (t - " + str(self.deltaT) + ")^" + format(self.n, d) + ")"
            
        else:
            if self.Ao > 0:
                return str(self.S[0]) + " * (1 + " + format(self.Ao, d) + " * t^" + format(self.n, d) + ")"
            else:
                return str(self.S[0]) + " * (1 - " + format(-self.Ao, d) + " * t^" + format(self.n, d) + ")"
        
        
    # --------------------------------------------------------
    # String para escrita da Power Law simplificada.
    # decimal: numero de casas decimais das constantes Ao e n
    # --------------------------------------------------------
    def LawSimple(self, decimal = 3):
        d = '.'+str(decimal)+'e'
        if self.deltaT:
            if self.Ao > 0:
                return str(self.S[0]) + " + " + format(self.S[0]*self.Ao, d) + " * (t - " + str(self.deltaT) + ")^" + format(self.n, d) + ")"
            else:
                return str(self.S[0]) + " - " + format(self.S[0]*-self.Ao, d) + " * (t - " + str(self.deltaT) + ")^" + format(self.n, d) + ")"

        else:
            if self.Ao > 0:
                return str(self.S[0]) + " + " + format(self.S[0]*self.Ao, d) + " * t^" + format(self.n, d)
            else:
                return str(self.S[0]) + " - " + format(self.S[0]*-self.Ao, d) + " * t^" + format(self.n, d)


    
    # --------------------------------------------------------
    # TAXA DE VARIACAO DA POWER LAW
    # d(S)/dt = So.n.Ao.tˆ(n-1)
    # --------------------------------------------------------
    def Law_dSdt(self):
        return self.S[0]*self.n*self.Ao, self.n-1
    
    
    # --------------------------------------------------------
    # TAXA DE VARIACAO DA POWER LAW
    # Derivada com elemento anterior
    # --------------------------------------------------------
    def ddt_ant(self):
        dp = (np.subtract(self.S[1:],self.S[:-1]))/(np.subtract(self.t[1:],self.t[:-1]))
        dt = self.t[1:]
        return dt,dp
    
    
    # --------------------------------------------------------
    # TAXA DE VARIACAO DA POWER LAW
    # Derivada com elemento posterior
    # --------------------------------------------------------
    def ddt_post(self):
        dp = (np.subtract(self.S[1:],self.S[:-1]))/(np.subtract(self.t[1:],self.t[:-1]))
        dt = self.t[:-1]
        return dt,dp

    # --------------------------------------------------------
    # TAXA DE VARIACAO DA POWER LAW
    # Derivada com elemento medio
    # --------------------------------------------------------
    def ddt_med(self):
        dp = (np.subtract(self.S[2:],self.S[:-2]))/(np.subtract(self.t[2:],self.t[:-2]))
        dt = self.t[1:-1]
        return dt,dp
    
    # --------------------------------------------------------
    # LINEAR FIT CONSTANTS
    # --------------------------------------------------------
    def Variables(self):
        return "Linear Fit Variables - Power Law:\n\n" + \
        "a (slope coef) = " + str(self.a)+ "\n"+ \
        "b (intercept coef) = " + str(self.b)+ "\n"+ \
        "r_value (correlation coefficient) = " + str(self.r_value)+ "\n"+ \
        "std_err (standard error) = " + str(self.std_err)+ "\n"


In [40]:
class ExponentialLaw:
    def __init__(self, t, S, deltaT = 0):
        import numpy as np
        from scipy import stats
        self.S = S
        self.t = t
        self.deltaT = deltaT
        if self.deltaT:
            for i in range (0,len(self.t)):
                self.t[i] -= self.deltaT
        
        # ----- Linearizacao dos Dados -----
        self.lin_S = []
        self.lin_t = []
        for i in range (1,len(self.S)):
            self.lin_S.append(np.log(self.S[i]/self.S[0]))
            self.lin_t.append(np.array(self.t[i]))
            

            
        # ----- LINEAR FIT -----

        # Biblioteca Estatistica
        slope, intercept, self.r_value, p_value, self.std_err = stats.linregress(self.lin_t, self.lin_S)
        # Biblioteca Numpy
        self.a, self.b = np.polyfit(np.array(self.lin_t), np.array(self.lin_S), deg=1)

        self.Ao = self.a
    
        if self.deltaT:
            for i in range (0,len(self.t)):
                self.t[i] += self.deltaT
        return
    
    
    
    # --------------------------------------------------------
    # Funcao para encontrar o tempo de um determinado acontecimento baseada na POWER-LAW
    # S(t) = So.exp(Ao.t) ----> t = ln((S/So))/Ao)
    # --------------------------------------------------------
    def Find_t(self, S):
        import numpy as np
        return (np.log(((np.array(S)/self.S[0]))/self.Ao)/self.Ao)+ self.deltaT
    
    # --------------------------------------------------------
    # Funcao para encontrar o S baseada na EXPONENTIAL-LAW
    # S(t) = So.exp(Ao.t)
    # --------------------------------------------------------
    def Find_S(self, t):
        import numpy as np
        S = self.S[0]*np.exp(self.Ao*(np.array(t)-self.deltaT))
        return S
    
    
    # --------------------------------------------------------
    # LEI EXPONENCIAL
    # S = So.exp(Ao.t)
    # Linearizacao
    # ln(S/So) = Ao.t
    # y = a.x
    # --------------------------------------------------------
    def LinearData(self):
        return self.lin_t,self.lin_S
    
    def LinearFit(self):
        self.LinearFit = []
        for i in range (len(self.lin_t)):
            self.LinearFit.append(self.a*self.lin_t[i] + self.b)
        return self.LinearFit
    
    
    
    
    # --------------------------------------------------------
    # String para escrita da Lei Exponencial.
    # decimal: numero de casas decimais da constante Ao
    # --------------------------------------------------------
    def Law(self, decimal = 3):
        d = '.'+str(decimal)+'e'
        if self.deltaT:
            return str(self.S[0]) + "*exp(" + format(self.Ao, d) + " *(t - " + str(self.deltaT) + "))"
        else:
            return str(self.S[0]) + "*exp(" + format(self.Ao, d) + " *t)"

            
    
    # --------------------------------------------------------
    # TAXA DE VARIACAO DA EXPONENCIAL
    # d(S)/dt = So.Ao.exp(Ao.t)
    # --------------------------------------------------------
    def Law_dSdt(self):
        return self.S[0]*self.Ao
    
    # --------------------------------------------------------
    # LINEAR FIT CONSTANTS
    # --------------------------------------------------------
    def Variables(self):
        return "Linear Fit Variables - Exponential Law:\n\n" + \
        "a (slope coef) = " + str(self.a)+ "\n"+ \
        "b (intercept coef) = " + str(self.b)+ "\n"+ \
        "r_value (correlation coefficient) = " + str(self.r_value)+ "\n"+ \
        "std_err (standard error) = " + str(self.std_err)+ "\n"
    

In [4]:
def StartLaw(t, S):
    import numpy as np
    
    # Derivate regarding previous element
    ds_1 = (np.subtract(S[1:],S[:-1]))/(np.subtract(t[1:],t[:-1]))
    dt_1 = t[1:]

    # Derivate regarding next element
    ds_2 = (np.subtract(S[1:],S[:-1]))/(np.subtract(t[1:],t[:-1]))
    dt_2 = t[:-1]

    # Derivate regarding mean of lateral elements
    ds_3 = (np.subtract(S[2:],S[:-2]))/(np.subtract(t[2:],t[:-2]))
    dt_3 = t[1:-1]
    
    # Possibilities of data rate variation
    first_data_value = []
    first_data_value.append(dt_1[np.nonzero(ds_1)[0][0]])
    first_data_value.append(dt_2[np.nonzero(ds_2)[0][0]])
    first_data_value.append(dt_3[np.nonzero(ds_3)[0][0]])
    
    first_data_index = np.where(np.array(t) >= min(first_data_value))[0][0]



    return ds_1, dt_1, ds_2, dt_2, ds_3, dt_3, first_data_index